<a href="https://colab.research.google.com/github/KIQVinicius/Data-Science/blob/master/AnomaliasJob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
import pandas as pd

In [110]:

from datetime import date, datetime,timedelta

In [111]:
dataset = pd.read_csv("MIPS1.csv")
dataset.head()

,Unnamed: 0,JOBNAME,DATE,Valor_MIPS,Porcentagem,Chave
0,0,FG921RID,08/06/2020,0.01,0.3,FG921RID08/06/2020
1,1,FG921RII,08/06/2020,0.01,0.3,FG921RII08/06/2020
2,2,FG921RIS,08/06/2020,0.01,0.3,FG921RIS08/06/2020
3,3,FG9227,08/06/2020,0.11,0.3,FG922708/06/2020
4,4,FG9227A,08/06/2020,1.67,0.3,FG9227A08/06/2020


In [112]:
porcentagem = float(0.001)/100 + 1
dataset.columns = ['Index', 'JOBNAME', 'DATE', 'Valor_MIPS', 'Porcentagem', 'Chave']

In [113]:
dataset = dataset.drop(columns=["Porcentagem", "Chave"], axis=1)

In [114]:
dataset['DATE'] = pd.to_datetime(dataset['DATE'])
dataset = dataset.sort_values(['DATE'])
first_day = dataset['DATE'].min()
last_day = dataset['DATE'].max()

In [115]:
first_day = datetime.strptime(str(first_day.month) + "/"+str(first_day.day) +"/"+str(first_day.year), '%d/%m/%Y')
last_day = datetime.strptime(str(last_day.month) + "/"+str(last_day.day) +"/"+str(last_day.year), '%d/%m/%Y')

In [116]:
first_day

datetime.datetime(2019, 11, 1, 0, 0)

In [117]:
last_day

datetime.datetime(2020, 6, 12, 0, 0)

In [118]:
dt = last_day.date() - first_day.date()
dt.days

224

In [119]:
weeks = int(dt.days/7)
weeks

32

In [120]:
months = int(dt.days/30)
months

7

In [121]:
data = first_day
dataset['JOBNAME'].value_counts()

CDSYSA      262
FS024597    262
FX2036A     262
FG94MRAW    262
PSIICHIN    262
           ... 
FG3006M1      1
FP524EX3      1
FG94B14       1
F6JP51EC      1
BS252142      1
Name: JOBNAME, Length: 15631, dtype: int64

In [122]:
frequencia = pd.DataFrame(dataset['JOBNAME'].value_counts())
frequencia.reset_index(inplace=True)
frequencia.columns = ['JOBNAME', 'Frequencia']
frequencia

,JOBNAME,Frequencia
0,CDSYSA,262
1,FS024597,262
2,FX2036A,262
3,FG94MRAW,262
4,PSIICHIN,262
...,...,...
15626,FG3006M1,1
15627,FP524EX3,1
15628,FG94B14,1
15629,F6JP51EC,1


In [123]:
frequencia = frequencia.query("Frequencia > 1")

In [124]:
dataset = dataset[dataset['JOBNAME'].isin(frequencia['JOBNAME'])]
dataset.head()

,Index,JOBNAME,DATE,Valor_MIPS
192478,192478,BD26B8L,2019-01-11,1.14
196089,196089,FS0245SI,2019-01-11,2.88
196088,196088,FS0245SF,2019-01-11,0.01
196087,196087,FS0245NL,2019-01-11,3.64
196086,196086,FS0245NG,2019-01-11,0.18


In [125]:
dataset = dataset.drop(columns=["Index"], axis=1)
dataset['JOBNAME'].value_counts()


FG94MRBM    262
FR32OK      262
FG95BR16    262
CFT         262
FG76MQRH    262
           ... 
FG930PIA      2
FZ686RTD      2
FG911ASA      2
FG93MOR1      2
FG9415SA      2
Name: JOBNAME, Length: 12923, dtype: int64

# Anomalia


In [128]:
def DetectarAnomalia(dataset, intervalo):
  if intervalo == "Mensal":
    periodo = months
    intervalo_de_dias = 30
  elif intervalo == "Semanal":
    periodo = weeks
    intervalo_de_dias = 7
  elif intervalo == "Quinzenal":
    periodo = int(dt.days/15)
    intervalo_de_dias = 15
  elif intervalo == "Trimestral":
    periodo = int(dt.days/90)
    intervalo_de_dias = 90
  elif intervalo == "Semestral":
    periodo = int(dt.days/180)
    intervalo_de_dias = 180
  elif intervalo == "Diario":
    periodo = int(dt.days)
    intervalo_de_dias = 1
  data = first_day
  tabela_anomalia = dataset.iloc[0:0]
  tabela_anomalia['Valor_MIPS_y'] = ""
  tabela_anomalia['Status'] = ""
  tabela_anomalia.columns = ['JOBNAME', 'DATE', 'Valor_MIPS_Atual', 'Valor_MIPS_Anterior', 'Status']
  dataset_semana_anterior = dataset.query("DATE >= " + " '"+ str(data)+ "' " + " and DATE < " + " '"+str(data+ timedelta(days = intervalo_de_dias)) + "' ")
  data = data+ timedelta(days = intervalo_de_dias)
  dataset_semana_anterior = dataset_semana_anterior[dataset_semana_anterior['Valor_MIPS']>0].groupby('JOBNAME')
  dataset_semana_anterior = pd.DataFrame(dataset_semana_anterior['Valor_MIPS'].mean())
  dataset_semana_anterior.reset_index(inplace =True)
  for i in range(0, periodo, 1):
    dataset_semana = dataset.query("DATE >= " + " '"+ str(data)+ "' " + " and DATE < " + " '"+str(data+ timedelta(days = intervalo_de_dias)) + "' ")
    #dataset = dataset.query("DATE >= " + " '"+ str(data+ timedelta(days = 30))+ "' ")
    #print("-------------------------------------------------")
    #dataset_semana = dataset_semana[dataset_semana['Valor_MIPS']>0].groupby('JOBNAME')
    #dataset_semana = pd.DataFrame(dataset_semana['Valor_MIPS'].mean())
    dataset_semana_anterior = dataset_semana_anterior[dataset_semana_anterior['JOBNAME'].isin(dataset_semana['JOBNAME'])]

    df = pd.merge(dataset_semana, dataset_semana_anterior, on = "JOBNAME", how ='inner', suffixes=['_Atual', '_Anterior'])
    df_aumentou = df[df['Valor_MIPS_Atual'] > df['Valor_MIPS_Anterior'] *porcentagem]
    df_aumentou['Status'] = "Aumentou"
    tabela_anomalia = tabela_anomalia.append(df_aumentou)
    #print("Tabela positiva: "+str(df_aumentou.shape[0]))
    df_diminuiu = df[df['Valor_MIPS_Atual']* porcentagem < df['Valor_MIPS_Anterior'] ]
    #print("Tabela negativa: "+str(df_diminuiu.shape[0]))
    df_diminuiu['Status'] = "Diminuiu"
    tabela_anomalia = tabela_anomalia.append(df_diminuiu)

    #print("Anomalias até " + str(data + timedelta(days = intervalo_de_dias) )[0:10]+": "  + str(tabela_anomalia.shape[0]))
    dataset_semana_anterior = dataset_semana[dataset_semana['Valor_MIPS']>0].groupby('JOBNAME')
    dataset_semana_anterior = pd.DataFrame(dataset_semana_anterior['Valor_MIPS'].mean())
    dataset_semana_anterior.reset_index(inplace=True)
    data = data+ timedelta(days = intervalo_de_dias)
  tabela_anomalia['Anomalia'] = intervalo
  return tabela_anomalia


In [129]:
tabela_diaria = DetectarAnomalia(dataset, "Diario")
tabela_final = tabela_diaria.iloc[0:0]
tabela_mensal = DetectarAnomalia(dataset, "Mensal")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [130]:
tabela_semanal = DetectarAnomalia(dataset, "Semanal")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [131]:
tabela_quinzenal = DetectarAnomalia(dataset, "Quinzenal")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [132]:
tabela_trimestral = DetectarAnomalia(dataset, "Trimestral")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [133]:
tabela_semestral = DetectarAnomalia(dataset, "Semestral")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [134]:
tabela_diaria = DetectarAnomalia(dataset, "Diario")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [135]:
tabela_final = tabela_final.append(tabela_mensal)
tabela_final = tabela_final.append(tabela_semanal)
tabela_final = tabela_final.append(tabela_quinzenal)
tabela_final = tabela_final.append(tabela_semestral)
tabela_final = tabela_final.append(tabela_diaria)
print(tabela_final.shape)

(2425904, 6)


In [136]:
tabela_final[tabela_final['Status'] == "Diminuiu"]

,JOBNAME,DATE,Valor_MIPS_Atual,Valor_MIPS_Anterior,Status,Anomalia
0,FG94F1II,2019-12-10,0.31,0.315,Diminuiu,Mensal
1,FG94F1II,2019-12-14,0.31,0.315,Diminuiu,Mensal
2,FG94F1II,2019-12-21,0.31,0.315,Diminuiu,Mensal
4,FG94F1IS,2019-12-10,0.31,0.325,Diminuiu,Mensal
5,FG94F1IS,2019-12-14,0.32,0.325,Diminuiu,Mensal
...,...,...,...,...,...,...
3237,FG94BM18,2020-06-06,0.01,0.130,Diminuiu,Diario
3239,FG94BM20,2020-06-06,0.46,0.570,Diminuiu,Diario
3240,FG94BM21,2020-06-06,0.03,21.700,Diminuiu,Diario
3245,FG94BR02,2020-06-06,0.01,0.060,Diminuiu,Diario
